# 1 All Models

References
- [LightGBM starter with feature engineering idea](https://www.kaggle.com/code/tommy1028/lightgbm-starter-with-feature-engineering-idea/notebook)

Goal: Aggregate Book data and Trade data to come up with a model to predict realized volatility.

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2'])/(df['bid_size2'] + df['ask_size2'])
    return wap

def log_returns(x):
    return np.log(x).diff()

def realized_volatility(x):
    return np.sqrt(np.sum(x**2))

def count_unique(x):
    return len(set(x))

In [2]:
def preprocess_book(file_path):
    df = pd.read_parquet(file_path)
    df['wap1'] = calc_wap1(df)
    df['returns1'] = df.groupby('time_id')['wap1'].apply(log_returns).values
    df['spread1'] = (df['ask_price1'] / df['bid_price1']) - 1

    df['wap2'] = calc_wap2(df)
    df['returns2'] = df.groupby('time_id')['wap2'].apply(log_returns).values
    df['spread2'] = (df['ask_price2'] / df['bid_price2']) - 1

    feature_map = {
        'wap1': ['mean', 'std'],
        'spread1': ['mean', 'std'],
        'returns1': [realized_volatility],
        'wap2': ['mean', 'std'],
        'spread2': ['mean', 'std'],
        'returns2': [realized_volatility],
    }
    
    """
    Improvements
    - Using the second most competitive buy level
    - Recency - use last few timesteps to improve prediction

    Questions
    - Does the inconsistent time difference affect the returns? 
    """

    df_feature = pd.DataFrame(df.groupby(['time_id']).agg(feature_map)).reset_index()
    df_feature.columns = ['_'.join(col).strip() for col in df_feature.columns.values]

    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(columns=['time_id_'], axis=1)

    return df_feature

book_example = preprocess_book('./data/book_train.parquet/stock_id=0')
book_example.head(5)

,wap1_mean,wap1_std,spread1_mean,spread1_std,returns1_realized_volatility,wap2_mean,wap2_std,spread2_mean,spread2_std,returns2_realized_volatility,row_id
0,1.003725,0.000693,0.000852,0.000212,0.004499,1.003661,0.000781,0.001178,0.000213,0.006999,0-5
1,1.000239,0.000262,0.000394,0.000157,0.001204,1.000206,0.000272,0.000671,0.000200,0.002476,0-11
2,0.999542,0.000864,0.000725,0.000164,0.002369,0.999680,0.000862,0.001121,0.000295,0.004801,0-16
3,0.998832,0.000757,0.000861,0.000280,0.002574,0.998633,0.000656,0.001160,0.000366,0.003637,0-31
4,0.999619,0.000258,0.000397,0.000130,0.001894,0.999626,0.000317,0.000697,0.000185,0.003257,0-62


In [3]:
def preprocess_trade(file_path):
    df = pd.read_parquet(file_path)

    feature_map = {
        'price' : ['mean', 'std'],
        'size' : ['sum', 'mean'],
        'order_count' : ['sum', 'mean'],
    }
    df_feature = pd.DataFrame(df.groupby('time_id').agg(feature_map).reset_index())
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]

    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(columns=['time_id_'], axis=1)
    return df_feature

trade_example = preprocess_trade('./data/trade_train.parquet/stock_id=0')
trade_example.head(5)

,price_mean,price_std,size_sum,size_mean,order_count_sum,order_count_mean,row_id
0,1.003722,0.000578,3179,79.475000,110,2.750000,0-5
1,1.000206,0.000304,1289,42.966667,57,1.900000,0-11
2,0.999204,0.000932,2161,86.440000,68,2.720000,0-16
3,0.999020,0.000729,1962,130.800000,59,3.933333,0-31
4,0.999618,0.000182,1791,81.409091,89,4.045455,0-62


# Train Dataset

In [12]:
from joblib import Parallel, delayed # parallel computing to save time

def preprocess(ids, is_train = True):
    df = pd.DataFrame()
    if is_train:
        book_path = './data/book_train.parquet'
        trade_path = './data/trade_train.parquet'
    else:
        book_path = './data/book_test.parquet'
        trade_path = './data/trade_test.parquet'

    for id in ids:
        f = f"stock_id={id}"
        book = preprocess_book(os.path.join(book_path, f))
        trade = preprocess_trade(os.path.join(trade_path, f))
        df_temp = pd.merge(book, trade, on='row_id', how='left')

        df = pd.concat([df, df_temp], axis=0)
    return df

train_features = preprocess([0] , is_train=True)
train_features.head(10)

,wap1_mean,wap1_std,spread1_mean,spread1_std,returns1_realized_volatility,wap2_mean,wap2_std,spread2_mean,spread2_std,returns2_realized_volatility,row_id,price_mean,price_std,size_sum,size_mean,order_count_sum,order_count_mean
0,1.003725,0.000693,0.000852,0.000212,0.004499,1.003661,0.000781,0.001178,0.000213,0.006999,0-5,1.003722,0.000578,3179,79.475000,110,2.750000
1,1.000239,0.000262,0.000394,0.000157,0.001204,1.000206,0.000272,0.000671,0.000200,0.002476,0-11,1.000206,0.000304,1289,42.966667,57,1.900000
2,0.999542,0.000864,0.000725,0.000164,0.002369,0.999680,0.000862,0.001121,0.000295,0.004801,0-16,0.999204,0.000932,2161,86.440000,68,2.720000
3,0.998832,0.000757,0.000861,0.000280,0.002574,0.998633,0.000656,0.001160,0.000366,0.003637,0-31,0.999020,0.000729,1962,130.800000,59,3.933333
4,0.999619,0.000258,0.000397,0.000130,0.001894,0.999626,0.000317,0.000697,0.000185,0.003257,0-62,0.999618,0.000182,1791,81.409091,89,4.045455
5,0.998605,0.000914,0.001639,0.000404,0.007902,0.998466,0.000951,0.001995,0.000507,0.010336,0-72,0.998137,0.001079,3395,130.576923,97,3.730769
6,0.996629,0.001862,0.001666,0.000446,0.010034,0.996725,0.001979,0.002158,0.000426,0.014493,0-97,0.996543,0.001800,2279,43.826923,114,2.192308
7,1.000064,0.001915,0.001040,0.000368,0.005331,1.000008,0.002036,0.001420,0.000337,0.006557,0-103,0.999806,0.001920,1181,42.178571,58,2.071429
8,1.001546,0.000636,0.000445,0.000146,0.001797,1.001583,0.000648,0.000722,0.000214,0.003536,0-109,1.001406,0.000714,1868,41.511111,85,1.888889
9,1.000142,0.000644,0.000468,0.000133,0.003273,1.000256,0.000688,0.000749,0.000183,0.005989,0-123,1.000012,0.000735,5135,80.234375,186,2.906250


In [13]:
train_targets = pd.read_csv('./data/train.csv')
train_ids = train_targets.stock_id.unique()
train_ids

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  26,  27,  28,
        29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  46,  47,  48,  50,  51,  52,  53,  55,  56,  58,
        59,  60,  61,  62,  63,  64,  66,  67,  68,  69,  70,  72,  73,
        74,  75,  76,  77,  78,  80,  81,  82,  83,  84,  85,  86,  87,
        88,  89,  90,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       118, 119, 120, 122, 123, 124, 125, 126])

In [21]:
train_targets['row_id'] = train_targets['stock_id'].astype(str) + '-' + train_targets['time_id'].astype(str)
train = pd.merge(train_targets[['row_id', 'target']], train_features, on='row_id', how='right')
train['stock_id'] = train['row_id'].apply(lambda x: x.split('-')[0])
train

,row_id,target,wap1_mean,wap1_std,spread1_mean,spread1_std,returns1_realized_volatility,wap2_mean,wap2_std,spread2_mean,spread2_std,returns2_realized_volatility,price_mean,price_std,size_sum,size_mean,order_count_sum,order_count_mean,stock_id
0,0-5,0.004136,1.003725,0.000693,0.000852,0.000212,0.004499,1.003661,0.000781,0.001178,0.000213,0.006999,1.003722,0.000578,3179,79.475000,110,2.750000,0
1,0-11,0.001445,1.000239,0.000262,0.000394,0.000157,0.001204,1.000206,0.000272,0.000671,0.000200,0.002476,1.000206,0.000304,1289,42.966667,57,1.900000,0
2,0-16,0.002168,0.999542,0.000864,0.000725,0.000164,0.002369,0.999680,0.000862,0.001121,0.000295,0.004801,0.999204,0.000932,2161,86.440000,68,2.720000,0
3,0-31,0.002195,0.998832,0.000757,0.000861,0.000280,0.002574,0.998633,0.000656,0.001160,0.000366,0.003637,0.999020,0.000729,1962,130.800000,59,3.933333,0
4,0-62,0.001747,0.999619,0.000258,0.000397,0.000130,0.001894,0.999626,0.000317,0.000697,0.000185,0.003257,0.999618,0.000182,1791,81.409091,89,4.045455,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,0-32751,0.002611,0.997938,0.000747,0.000552,0.000202,0.002579,0.997864,0.000769,0.000818,0.000188,0.003821,0.997556,0.000714,3450,66.346154,159,3.057692,0
3826,0-32753,0.001190,1.000310,0.000551,0.000543,0.000147,0.002206,1.000487,0.000599,0.000806,0.000128,0.002847,1.000325,0.000479,4547,162.392857,109,3.892857,0
3827,0-32758,0.004264,0.999552,0.000743,0.000525,0.000244,0.002913,0.999456,0.000736,0.000811,0.000307,0.003266,0.999633,0.000695,4250,118.055556,126,3.500000,0
3828,0-32763,0.004352,1.002357,0.000356,0.000480,0.000145,0.003046,1.002386,0.000424,0.000759,0.000206,0.005105,1.002305,0.000342,3217,60.698113,114,2.150943,0


In [ ]:
test_features = preprocess([0], is_train=False)
test_target = pd.read_csv('./data/test.csv')
test = pd.merge(test_target[['row_id', ]], test_features, on='row_id', how='right')
test

,stock_id,time_id,row_id,wap1_mean,wap1_std,spread1_mean,spread1_std,returns1_realized_volatility,wap2_mean,wap2_std,spread2_mean,spread2_std,returns2_realized_volatility,price_mean,price_std,size_sum,size_mean,order_count_sum,order_count_mean
0,0,4,0-4,1.000405,0.00017,0.000557,0.000028,0.000294,1.00055,0.000153,0.001066,0.000142,0.000252,1.000151,0.000168,201,67.0,11,3.666667


In [ ]:
df_train = train.copy()

stock_id_target_mean = df_train.groupby('stock_id')['target'].mean()
df_test['stock_id_target_enc'] = df_test['stock_id'].map(stock_id_target_mean)

,row_id,target,wap1_mean,wap1_std,spread1_mean,spread1_std,returns1_realized_volatility,wap2_mean,wap2_std,spread2_mean,spread2_std,returns2_realized_volatility,price_mean,price_std,size_sum,size_mean,order_count_sum,order_count_mean
0,0-5,0.004136,1.003725,0.000693,0.000852,0.000212,0.004499,1.003661,0.000781,0.001178,0.000213,0.006999,1.003722,0.000578,3179,79.475000,110,2.750000
1,0-11,0.001445,1.000239,0.000262,0.000394,0.000157,0.001204,1.000206,0.000272,0.000671,0.000200,0.002476,1.000206,0.000304,1289,42.966667,57,1.900000
2,0-16,0.002168,0.999542,0.000864,0.000725,0.000164,0.002369,0.999680,0.000862,0.001121,0.000295,0.004801,0.999204,0.000932,2161,86.440000,68,2.720000
3,0-31,0.002195,0.998832,0.000757,0.000861,0.000280,0.002574,0.998633,0.000656,0.001160,0.000366,0.003637,0.999020,0.000729,1962,130.800000,59,3.933333
4,0-62,0.001747,0.999619,0.000258,0.000397,0.000130,0.001894,0.999626,0.000317,0.000697,0.000185,0.003257,0.999618,0.000182,1791,81.409091,89,4.045455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,0-32751,0.002611,0.997938,0.000747,0.000552,0.000202,0.002579,0.997864,0.000769,0.000818,0.000188,0.003821,0.997556,0.000714,3450,66.346154,159,3.057692
3826,0-32753,0.001190,1.000310,0.000551,0.000543,0.000147,0.002206,1.000487,0.000599,0.000806,0.000128,0.002847,1.000325,0.000479,4547,162.392857,109,3.892857
3827,0-32758,0.004264,0.999552,0.000743,0.000525,0.000244,0.002913,0.999456,0.000736,0.000811,0.000307,0.003266,0.999633,0.000695,4250,118.055556,126,3.500000
3828,0-32763,0.004352,1.002357,0.000356,0.000480,0.000145,0.003046,1.002386,0.000424,0.000759,0.000206,0.005105,1.002305,0.000342,3217,60.698113,114,2.150943


## KFold Cross Validation

In [15]:
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold, StratifiedGroupKFold, RepeatedKFold

random_state = 42

kf = KFold(n_splits = 10, shuffle=True,random_state = random_state)
kf

KFold(n_splits=10, random_state=42, shuffle=True)